# A Template-Based Process for Agent-Based Modeling through Generalized Dynamical Systems in cadCAD
by: Octopus

## Contemporary Research Problems Need New Approaches

A research organization dealing with problems in a large complex systems faces many challenges: 

**Challenge 1:** We often need to model complex systems that have large numbers of *agents*: autonomous individuals who make decisions based on their knowledge at a given time. In modeling systems with agents, it is tempting to use an approach that reduces complexity. For instance, a modeler might aggregate individuals into a collective approximation, or replace the logic of agent decisions with random samples from a probability distribution. While these approaches are used to great effect in traditional engineering, they risk wahing away the most extreme risks the system might face. 

**Challenge 2:** Modeling teams often need to rely on individuals with diverse skillsets and schedule availabilities. Communication can be hampered by translating between domain terminologies. The process of developing a model, before translating it into code, then waiting on the coder to develop an initial version prior to validation, can introduce substantial delays as team mebers wait on each other to finish. Designing a workflow based on modular pieces that allow for parallel work can greatly increase efficiency.

Our goal in this notebook is to lay out a framework that addresses both challenges uses *Agent-Based Thinking*, using principles from Generalized Dynamical Systems and Object-Oriented programming for a modeling approach implemented in cadCAD. 

In [1]:
from abc import ABC
from dataclasses import dataclass

from typing import (Any, 
                    Callable, 
                    Dict, 
                    Literal, 
                    Optional, 
                    TypedDict)

import pandas as pd
import random

## Agent Decisions in GDS and cadCAD Models

### Overview
At a high level, our solution depends on formatting cadCAD models for multi-stage game models in a standard way. In this format, 
* each stage of the game corresponds to a specific partial state update block in cadCAD. 
* any computation that needs to be done inside a policy function, is passed to an agent to a standard decision. 


### Running Example: The Killer Tree Game
We will use a running example of the [Killer Tree Model](TODO: add link) introduced by Koller and Milch. In this model, there are two agents, **Alice** and **Bob**. Alice and Bob are next-door neighbors. **Bob** has a large **Tree** in his yard, which he would like to keep forever. **Alice** would like to build a patio for her house, but she will only enjoy the view if Bob's tree is removed. We can view this situation as a game with the following stages:
* Stage 1: **Alice** decides whether or not to **poison the tree**.
* Stage 2: The **Tree** may become sick.
* Stage 3: **Bob** can see whether or not the tree is sick. **Bob** decides whether or not to **call a tree doctor**. **Alice** can see that the tree doctor was called, but does not know why. 
* Stage 4: Based on her available knowledge **Alice** decides whether or not to build the deck. The **Tree** may become dead. 

### Translating Sequential Game to cadCAD Structure

The stages of the game naturally translate to partial state update blocks in cadCAD. 
Here is one way we might lay out the overview of the game. 

```python
cadCAD_model  = [
    {
        'label': 'Alice: Poison Tree Decision' # Stage 0
        'policies': {
            'alice:poison_tree':p_alice_poison_decision,
        },
        'variables': {
            'first_variable': s_tree_poisoned,
        }
    },
    {
        'label': 'Does The Tree Become Sick?', # Stage 1
        'policies': {
            'tree_sick_process': p_tree_sick_process
        },
        'variables': {
            'tree_sick': s_tree_sick
        }
    },
    {
        'label': 'Is The Tree Doctor Called?', # Stage 2
        'policies': {
            'call_doctor_decision': p_bob_doctor_decision
        },
        'variables': {
            'doctor_called': s_doctor_called 
        }
    },
    {
        'label': 'Final Outcome', # Stage 3
        'policies': {
            'build_deck_decision': p_alice_deck_decision,
            'tree_dies_process': p_tree_dies_process
         },
        'variables': {
            'deck_built': s_deck_built,
            'tree_dead': s_tree_dead
        }
    }
]
```

Creating this structure follows directly from a Multi-Agent Influence Diagram, and requires no coding knowledge at all.

### Creating Policy and State Update Functions for Games

For this type of multi-stage game, every state variable is updated either as a result of an **agent decision** or **chance process**. However, we can use the same framework for either type, since both agent decisions and chance processes can be described through conditional probability distributions. This follows the convention (TODO: check spelling of Harsinyi), who emphasized "Nature" as a player in the game. For the Killer Tree model, we will have *Tree* as an agent.  

With this setup, we can have all agent decisions follow a similar process:
1. Agent captures available information.
2. Agent chooses the appropriate *decision rule* for the circumstance, from their predefined *strategy*. 
3. Agent creates a *decision signal* to send to update the state varible.

In actual code, the template looks something like this:
```python
# Policy Function
def p_agent_decision(params, substep, state_history, previous_state, _input):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "agent_name"
    decision_name = "decision_name"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_strategy.get(decision_name)
    decision_made = decision_rule()

    return {decision name: decision} 

# State Update Function
def s_var_name(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "decision_name"
    var_name = "var_name"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)
``` 

From a coding style perspective, this emphasizes the ability to re-use the structure as an easily-readable template. This structure can easily be filled in by someone who is not a master hacker, as long as they are familiar with the model basics: which state variables exist, which decisions update those variables, and which agents make those decisions. 

### Game State: 

## Python Class Structure for Individual Agents

The overall model structure and associated functions are designed to be as amenable as possible to diverse skill-sets. However, at some point we need to write the granular logic of actually processing (filtering) available information (inocming signals) to reach a final decision (outgoing signal). That time is now. 


**Players** interact to update the state of **Games**, which have `States` and `Params`, both of which can be given as `Typed Dicts`. 


A **Player** consists of:
- a name
- a description
- strategy, which encapsulates decision rules
- utility functions

Here is a general code design that we can use: 


In [2]:
PlayerRules = TypedDict
PlayerValues = TypedDict

@dataclass
class PlayerStrategy(ABC):
    name: str 
    rules: PlayerRules

@dataclass 
class PlayerUtility(ABC):
    name: str 
    values: PlayerValues

@dataclass
class Player(ABC):
    name: str
    description: str
    strategy: PlayerStrategy
    utility: PlayerUtility

# Create dictionaries for GameState and GameParams
GameState = TypedDict 
GameParams = TypedDict



Let's do a specific example to see how this general pattern enables experimentation. 

When we go to implementan actual game state, it will look like this: 

In [3]:
@dataclass
class KillerTreeGameState(GameState):
    tree_is_poisoned: Optional[Literal[0, 1]]
    tree_is_sick: Optional[Literal[0, 1]]

    doctor_is_called: Optional[Literal[0, 1]]
    deck_is_built: Optional[Literal[0, 1]]

    tree_is_dead: Optional[Literal[0, 1]]



Two comments may be valuable here: 
1. This design is completely oriented towards parameter sweeps: we would want to sweep over different versions of the players, while the state variables will always be the same.
2. In future iterations, it might be worth using a data type that signals more clearly that the Tree is not the same kind of `Player` as Alice and Bob. 

In [4]:
@dataclass
class AliceRules(PlayerRules):
    poison_tree: Callable[Dict, Literal[0,1]]
    build_deck: Callable[Dict, Literal[0,1]]
    
@dataclass
class AliceStrategy(PlayerStrategy):
    name: str = "default strategy"
    rules: AliceRules = None

@dataclass
class AliceValues(PlayerValues):
    poison_tree: Callable[Dict, float]
    tree_is_dead: Callable[Dict, float]

@dataclass
class AliceUtility(PlayerUtility):
    name: str = "default utility"
    values: AliceValues = None
 
@dataclass    
class AlicePlayer(Player):
    name: str = "Alice"
    description: str = "The generic version of Alice. Please replace with a custom Alice."
    strategy: AliceStrategy = None
    utility: AliceUtility = None


In [5]:
@dataclass
class BobRules(PlayerRules):
    call_doctor: Callable[Any, Literal[None, 0,1]]

@dataclass
class BobValues(PlayerValues):
    call_doctor: Callable[Any, Literal[None, 0,1]]
    tree_is_dead: Callable[Any, Literal[None, 0, 1]]


@dataclass
class BobStrategy(PlayerStrategy):
    name = "generic bob strategy"
    rules: BobRules = None

@dataclass
class BobUtility(PlayerUtility):
    name = "generic bob values"
    values: BobValues = None

@dataclass    
class BobPlayer(Player):
    name: str = "Bob"
    description: str = "The generic version of Bob. Please replace with a custom Bob."
    strategy: BobStrategy = None
    utility: BobUtility = None


In [6]:
@dataclass
class TreeRules(PlayerRules):
    become_sick: Callable[Any, Literal[None, 0,1]]
    become_dead: Callable[Any, Literal[None, 0,1]]

@dataclass
class TreeStrategy(PlayerStrategy):
    name: str = "Generic Tree Strategy"
    rules: TreeRules = None
    
@dataclass    
class TreePlayer(Player):
    name: str = "Tree"
    description: str = "The generic version of Tree. Please replace with a custom Tree."
    strategy: TreeStrategy = None
    utility: PlayerUtility = None

## Let's Actually Play a Game

So far we just have abstract versions of **Alice**, **Bob**, and **Tree**. To have versions that can actually play, we need to give them strategies and utilities. 



### Building an Alice

In [7]:
def alice_always_poison_tree(available_info) -> Literal[0,1]:
    return 1

def alice_safe_build_deck(available_info) -> Literal[0,1]:
    tree_is_poisoned = available_info.get("previous_state").get("tree_is_poisoned")
    doctor_is_called = available_info.get("previous_state").get("doctor_is_called")
    build_deck_decision = tree_is_poisoned * doctor_is_called
    return build_deck_decision

cautiously_aggressive_alice_rules = AliceRules(poison_tree = alice_always_poison_tree,
                                         build_deck = alice_safe_build_deck)

cautiously_aggressive_alice_strategy = AliceStrategy(name = "cautiously aggressive",
                                                     rules = cautiously_aggressive_alice_rules)

def poison_tree_zero_cost(available_info) -> float:
    value = 0
    return value 

def view_cost(available_info) -> float:
    deck_built = avaiable_info.get("previous_state").get("deck_is_built")
    tree_is_dead = available_info.get("previous_state").get("tree_is_dead")
    value = -50 * deck_built + 100 * tree_is_dead
    return value

only_care_about_deck_values = AliceValues(poison_tree = poison_tree_zero_cost,
                                         build_deck = view_cost)
only_care_about_deck_utility = AliceUtility(name = "only care about deck",
                                              values = only_care_about_deck_values)

example_alice = AlicePlayer(name = "example alice",
                          description = "Build deck if doctor is called",
                          strategy = cautiously_aggressive_alice_strategy,
                          utility = only_care_about_deck_utility )



### Building a Bob 

In [8]:
def bob_always_call_doctor(available_info) -> Literal[0,1]:
    return 1

safe_bob_rules = BobRules(call_doctor = bob_always_call_doctor)

safe_bob_strategy = BobStrategy(name = "always_call_doctor",
                                rules = safe_bob_rules)

def bob_free_doctor(available_info) -> float:
    value = 0
    return value

def bob_loves_tree(available_info) -> float: 
    tree_is_dead = available_info.get("previous_state").get("tree_is_dead")
    value = -1000 * tree_is_dead
    return value
    
safe_bob_utility = BobValues(call_doctor = bob_free_doctor,
                           tree_is_dead = bob_loves_tree)


example_bob = BobPlayer(name = "example bob",
                        description = "bob plays it safe",
                        strategy = safe_bob_strategy,
                        utility = safe_bob_utility
                        )

## Creating a Tree

In [9]:

def tree_sick_rule(available_info) -> int:
   tree_is_poisoned = available_info.get("previous_state").get("tree_is_poisoned")
   become_sick_threshold = 1.0
   random_num = random.random()
   tree_becomes_sick = int(random_num < become_sick_threshold)

   return tree_becomes_sick

def tree_dead_rule(available_info) -> int:
    tree_is_sick = available_info.get("previous_state").get("tree_is_sick")
    doctor_is_called = available_info.get("previous_state").get("doctor_is_called")
    tree_becomes_dead = tree_is_sick * (1 - doctor_is_called)
    return tree_becomes_dead

my_tree_rules = TreeRules(become_sick = tree_sick_rule,
                          become_dead = tree_dead_rule)

my_tree_strategy = TreeStrategy(name = "example", 
                                rules = my_tree_rules)

my_tree = TreePlayer(name = "my tree",
                    description = "there are many like it, but this one is mine",
                    strategy = my_tree_strategy)

In [10]:
# Stage 1: Alice Decides Whether or Not to Poison Tree

# Alice Poison Tree Decision 
def p_alice_poison_decision(params, substep, state_history, previous_state):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "alice"
    decision_name = "poison_tree"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_decision_rules.get(decision_name)
    decision_made = decision_rule(available_info)

    return {decision_name: decision_made} 

# Update Tree Is Poisoned Variable
def s_tree_poisoned(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "poison_tree"
    var_name = "tree_is_poisoned"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

# Stage 2: Does Tree Become Sick? 

# Alice Poison Tree Decision 
def p_tree_become_sick_decision(params, substep, state_history, previous_state, _input):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "tree"
    decision_name = "become_sick"

    
    try:
        #Step 3: Access the relevant decision rule and make the decision
        agent_decision_rules = params[agent_name].strategy.rules
        decision_rule = agent_decision_rules.get(decision_name)
        decision_made = decision_rule(available_info)

    except:
        print("For available info")
        for key, val in available_info.items():
            print(f"The value of {key} is {val}.")

    return {decision_name: decision_made} 

# Update Tree Is Poisoned Variable
def s_tree_poisoned(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "poison_tree"
    var_name = "tree_is_poisoned"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

# Stage 2: Does Tree Become Sick? 

# Tree Become Sick  Decision 
def p_tree_become_sick(params, substep, state_history, previous_state):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "tree"
    decision_name = "become_sick"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_decision_rules.get(decision_name)
    decision_made = decision_rule(available_info)

    return {decision_name: decision_made} 

# Update Tree Is Sick Variable
def s_tree_sick(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "become_sick"
    var_name = "tree_is_sick"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

# Stage 3: Is Doctor Called, Is Deck Built?

# Policy Function: Bob Call Doctor
def p_bob_call_doctor(params, substep, state_history, previous_state):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "bob"
    decision_name = "call_doctor"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_decision_rules.get(decision_name)
    decision_made = decision_rule(available_info)

    return {decision_name: decision_made} 

# Policy Function: Alice Build Deck
def p_alice_build_deck(params, substep, state_history, previous_state):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "alice"
    decision_name = "build_deck"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_decision_rules.get(decision_name)
    decision_made = decision_rule(available_info)

    return {decision_name: decision_made} 

# State Update Function: Is Doctor Called?
def s_is_doctor_called(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "call_doctor"
    var_name = "doctor_is_called"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

# State Update Function: Is Deck Built?
def s_is_deck_built(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "build_deck"
    var_name = "deck_is_built"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

# Stage 4: Does Tree Die?
def p_tree_become_dead(params, substep, state_history, previous_state):
    #Step 1: Take information that was passed in
    available_info = locals()

    #Step 2: Define the agent name and the decision to be made
    agent_name = "tree"
    decision_name = "become_dead"

    #Step 3: Access the relevant decision rule and make the decision
    agent_decision_rules = params[agent_name].strategy.rules
    decision_rule = agent_decision_rules.get(decision_name)
    decision_made = decision_rule(available_info)

    return {decision_name: decision_made} 

def s_is_tree_dead(params, substep, state_history, previous_state, _decision_signal):
    decision_name = "become_dead"
    var_name = "tree_is_dead"
    new_value = _decision_signal.get(decision_name)

    return (var_name, new_value)

In addition to the above functions, we will build things that reset the value of state variables to `None`, so we can start fresh each time. 



In [11]:
def p_game_reset_vars(params, substep, state_history, previous_state):
    return {"x0": 1} 

def s_reset_tree_is_poisoned(params, substep, state_history, previous_state, _decision_signal):
    var_name = "tree_is_poisoned"
    new_value = None

    return (var_name, new_value)

def s_reset_tree_is_sick(params, substep, state_history, previous_state, _decision_signal):
    var_name = "tree_is_sick"
    new_value = None

    return (var_name, new_value)


def s_reset_doctor_is_called(params, substep, state_history, previous_state, _decision_signal):
    var_name = "doctor_is_called"
    new_value = None

    return (var_name, new_value) 

def s_reset_deck_is_built(params, substep, state_history, previous_state, _decision_signal):
    var_name = "deck_is_built"
    new_value = None

    return (var_name, new_value) 
 

def s_reset_tree_is_dead(params, substep, state_history, previous_state, _decision_signal):
    var_name = "tree_is_dead"
    new_value = None

    return (var_name, new_value)  

In [12]:
# Translating model from earlier to partial state update blocks. 

psubs  = [
    {
    'label': 'Game: Reset All Variables', # Stage 0
    'policies': {
        'game_reset_all_vars': p_game_reset_vars,
        },
    'variables': {
        'tree_is_poisoned': s_reset_tree_is_poisoned,
        'tree_is_sick': s_reset_tree_is_sick,
        'doctor_is_called': s_reset_doctor_is_called,
        'deck_is_built': s_reset_deck_is_built,
        'tree_is_dead': s_reset_tree_is_dead
        },
    },
    {
        'label': 'Alice: Poison Tree Decision', # Stage 1
        'policies': {
            'alice_poison_tree': p_alice_poison_decision,
        },
        'variables': {
            'tree_is_poisoned': s_tree_poisoned,
        }
    },
    {
        'label': 'Does The Tree Become Sick?', # Stage 2
        'policies': {
            'tree_become_sick': p_tree_become_sick,
        },
        'variables': {
            'tree_is_sick': s_tree_sick
        }
    },
    {
        'label': 'Is The Tree Doctor Called?', # Stage 3
        'policies': {
            'call_doctor_decision': p_bob_call_doctor,
        },
        'variables': {
            'doctor_is_called': s_is_doctor_called 
        }
    },
    {
        'label': 'Final Outcome', # Stage 4
        'policies': {
            'tree_becomes_dead': p_tree_become_dead,
            'build_deck_decision': p_alice_build_deck,

         },
        'variables': {
            'tree_is_dead': s_is_tree_dead,
            'deck_is_built': s_is_deck_built
        }
    }
]

## Setting Up a cadCAD Model 

In [13]:
from cadCAD.configuration import Configuration, Experiment  # type: ignore
from cadCAD.configuration.utils import config_sim  # type: ignore
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor  # type: ignore
from cadCAD.tools.utils import add_parameter_labels

In [14]:
    N_samples = 1
    N_timesteps = 10
    
    simulation_parameters = {"N": N_samples, 
                             "T": range(N_timesteps),
                              "M": {'bob': [example_bob],
                                    'alice': [example_alice],
                                    'tree': [my_tree]} }
    sim_config = config_sim(simulation_parameters)

In [15]:
initial_state = KillerTreeGameState(
    tree_is_poisoned = None,
    tree_is_sick = None,
    doctor_is_called = None,
    deck_is_built= None,
    tree_is_dead = None)

In [16]:
# Create a new experiment
exp = Experiment()
exp.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=psubs,
)
configs = exp.configs

_exec_mode = ExecutionMode().local_mode

exec_context = ExecutionContext(
            _exec_mode, 
            additional_objs={"deepcopy_off": True}                            
        )
executor = Executor(
    exec_context=exec_context, 
    configs=configs,
    supress_print=True
)

# Execute the cadCAD experiment
(records, tensor_field, _) = executor.execute()

# Parse the output as a pandas DataFrame
df = pd.DataFrame(records)
 

In [17]:
df

,tree_is_poisoned,tree_is_sick,doctor_is_called,deck_is_built,tree_is_dead,simulation,subset,run,substep,timestep
0,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0
1,NaN,NaN,NaN,NaN,NaN,0,0,1,1,1
2,1.0,NaN,NaN,NaN,NaN,0,0,1,2,1
3,1.0,1.0,NaN,NaN,NaN,0,0,1,3,1
4,1.0,1.0,1.0,NaN,NaN,0,0,1,4,1
5,1.0,1.0,1.0,1.0,0.0,0,0,1,5,1
6,NaN,NaN,NaN,NaN,NaN,0,0,1,1,2
7,1.0,NaN,NaN,NaN,NaN,0,0,1,2,2
8,1.0,1.0,NaN,NaN,NaN,0,0,1,3,2
9,1.0,1.0,1.0,NaN,NaN,0,0,1,4,2


## Conclusion

This is a proof-of-concept on making object-oriented principles interoperable with cadCAD. Hopefully we (or someone else) will be back to extend it later! 